In [1]:
import gym
import numpy as np
import csv
import json

In [2]:
import torch
import torch.nn as nn
from scipy.special import softmax

In [ ]:
RENDER = False

In [3]:
class ValueNet(nn.Module):
    def __init__(self, input_dim, hidden, output_dim):
        super(ValueNet, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, output_dim),
        )
    
    def forward(self, x):
        return self.model(x)

In [4]:
class MLP(nn.Module):
    def __init__(self,
                 input_dim : int,
                 hidden : int,
                 output_dim : int):
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden)
        self.l2 = nn.Linear(hidden, hidden)
        self.l3 = nn.Linear(hidden, output_dim)
        
        nn.init.xavier_uniform_(self.l1.weight)
        nn.init.xavier_uniform_(self.l2.weight)
        nn.init.xavier_uniform_(self.l3.weight)
        
    def forward(self, inputs):
        x = self.l1(inputs)
        x = nn.functional.tanh(x)
        x = self.l2(x)
        x = nn.functional.tanh(x)
        x = self.l3(x)
        return x

In [5]:
class ModelWithPrior(nn.Module):
    def __init__(self,
                 base_model : nn.Module,
                 prior_model : nn.Module,
                 prior_scale : float = 1.0):
        super().__init__()
        self.base_model = base_model
        self.prior_model = prior_model
        self.prior_scale = prior_scale
        
    def forward(self, inputs):
        with torch.no_grad():
            prior_out = self.prior_model(inputs)
            prior_out = prior_out.detach()
        model_out = self.base_model(inputs)
        return model_out + (self.prior_scale * prior_out)

In [6]:
class POLO(object):
    def __init__(self, env, K, T, U, lambda_, noise_mu, 
                 noise_sigma, u_init, memory_size, observation_space, action_space, state_space, 
                 net_hidden_layers, num_nets, state_samples, gradient_steps, gamma=0.99, log_file=None, 
                 noise_gaussian=True):
        
        self.memory_size = memory_size
        self.obs_mem = np.zeros((self.memory_size, observation_space))
        self.state_mem = [None for i in range(self.memory_size)]
        self.targets_mem = np.zeros((self.memory_size, num_nets))
        
        self.num_nets = num_nets
        
        self.K = K  # N_SAMPLES
        self.T = T  # TIMESTEPS
        self.lambda_ = lambda_
        self.noise_mu = noise_mu
        self.noise_sigma = noise_sigma
        self.U = U
        self.u_init = u_init
        self.reward_total = np.zeros(shape=(self.K))
        
        self.state_samples = state_samples
        self.gradient_steps = gradient_steps

        self.env = env

        ############################
        if self.env.unwrapped.spec.id == "Pendulum-v0":
            self.x_init = self.env.env.state
        elif self.env.unwrapped.spec.id == "HumanoidStandup-v2":
            self.x_init = env.sim.get_state()
        ############################
        
        self.gamma = gamma
        
        self.max_reward_for_net = np.full((self.num_nets), '-inf', dtype=np.float)
        
        self.log_file = log_file
        if log_file is not None:
            self.writer = csv.writer(log_file, delimiter='\t')
            headers = ["timestamp", "reward", "action", "state"]
            self.writer.writerow(headers)
        

        if noise_gaussian:
            self.noise = np.random.normal(loc=self.noise_mu, scale=self.noise_sigma, size=(self.K, self.T, self.env.action_space.shape[0]))
        else:
            self.noise = np.full(shape=(self.K, self.T), fill_value=0.9)
            
        self._build_value_nets(observation_space, net_hidden_layers, 1)
        
    def _build_value_nets(self, input_dim, hidden, output_dim):
        self.value_nets = []
        self.loss_funcs = []
        self.optimizers = []
        
        for i in range(self.num_nets):
#             self.value_nets.append(ValueNet(input_dim, hidden, output_dim))
            self.value_nets.append(ModelWithPrior(MLP(input_dim, hidden, output_dim), 
                                                  MLP(input_dim, hidden, output_dim)))
            self.loss_funcs.append(nn.MSELoss())
            self.optimizers.append(torch.optim.Adam(self.value_nets[-1].parameters(), lr=0.01))

    def _get_reward_from_state(self, s):
        root_z = s[0]
        if root_z > 1.1:
            return 1.0
        else:
            return 1.0 - (1.1 - root_z)

    def learn(self, env):
        for _ in range(self.gradient_steps):
            sampled_idx = np.random.choice(np.min([self.memory_counter, self.memory_size]), size=self.state_samples, replace=False)
            
            sampled_obs = self.obs_mem[sampled_idx,:]
            sampled_targets = self.targets_mem[sampled_idx,:]
            sampled_targets = sampled_targets.transpose()
            
            for i in range(self.num_nets):
                net = self.value_nets[i]
                loss_func = self.loss_funcs[i]
                optimizer = self.optimizers[i]
                
                optimizer.zero_grad()
                
#                 print("Target: {}".format(sampled_targets[i]))
                preds = net(torch.tensor(sampled_obs, dtype=torch.float))
                
                target = torch.tensor([sampled_targets[i]], dtype=torch.float)
                loss = loss_func(preds, target)
            

                loss.backward()
                optimizer.step()
                
                
#             print("=====")
#             print(sampled_obs)
#             print(sampled_targets)
        return
        self.x_init = self.env.sim.get_state()
        
        for _ in range(self.gradient_steps):
            sampled_idx = np.random.choice(np.min([self.memory_counter, self.memory_size]), size=self.state_samples, replace=False)

    #             print(self.state_mem)
    #             print(idx)
    #             sampled_states = self.state_mem[idx]

            sampled_obs = self.obs_mem[sampled_idx,:]

    #             sampled_obs = []

            targets = [None for i in range(self.num_nets)]

            for index in sampled_idx:
                s_state = self.state_mem[index]
                o = self.state_mem[index]
    #             for s_state, o in zip(sampled_states, sampled_obs):


                max_rewards = [float('-inf') for _ in range(self.num_nets)]

                for k in range(self.K):
    #                     print(len(self.x_init), self.x_init)
    #                     print(len(s_state), s_state)
                    self.env.sim.set_state(s_state)
                    discount = 1
                    total_reward = 0
                    for t in range(self.T):
                        perturbed_action_t = self.U[t] + self.noise[k, t]

                        s, reward, _, _ = env.step(np.array([perturbed_action_t]))

                        total_reward += discount * reward
                        discount *= self.gamma

                    for i in range(self.num_nets):
                        net = self.value_nets[i]
                        reward_for_net = torch.tensor(total_reward, dtype=torch.float) + net(torch.tensor(s[:22], dtype=torch.float))
                        if reward_for_net > max_rewards[i]:
                            max_rewards[i] = reward_for_net



                for i in range(self.num_nets):
                    target = max_rewards[i]

                    if targets[i] is None:
                        targets[i] = torch.tensor([[target]], dtype=torch.float)
                    else:
                        targets[i] = torch.cat((targets[i], torch.tensor([[target]], dtype=torch.float)))

        
#         for _ in range(self.gradient_steps):
            for i in range(self.num_nets):
                net = self.value_nets[i]
                loss_func = self.loss_funcs[i]
                optimizer = self.optimizers[i]

                optimizer.zero_grad()

                preds = net(torch.tensor(sampled_obs, dtype=torch.float))

                loss = loss_func(preds, targets[i])

                loss.backward()
                optimizer.step()
                
                
        self.env.sim.set_state(self.x_init)
        
    def get_aggregated_value(self, values):
        weights = softmax(0.01 * values)
        weighted_values = values * weights
        
        return sum(weighted_values)
    
    def get_network_values(self, s):
        values = []
        for net in self.value_nets:
            values.append(net(torch.FloatTensor(s)).tolist())
            
        values = np.array(values)
        return values
    
    def _compute_total_reward(self, k):
        discount = 1
        ############################
        if self.env.unwrapped.spec.id == "Pendulum-v0":
            self.env.env.state = self.x_init
        elif self.env.unwrapped.spec.id == "HumanoidStandup-v2":
            self.env.sim.set_state(self.x_init)
        ############################
        for t in range(self.T):
            perturbed_action_t = self.U[t] + self.noise[k, t]
            s, reward, _, _ = self.env.step(np.array([perturbed_action_t]))
            if self.env.unwrapped.spec.id == "HumanoidStandup-v2":
                reward = self._get_reward_from_state(s)
            self.reward_total[k] += discount * reward
            discount *= self.gamma
        
        network_values = self.get_network_values(s[:22])
        
        for i in range(self.num_nets):
            reward_for_net = self.reward_total[k] + discount * network_values[i]
            if reward_for_net > self.max_reward_for_net[i]:
                self.max_reward_for_net[i] = reward_for_net
        
        self.reward_total[k] += discount * self.get_aggregated_value(network_values)

    def _ensure_non_zero(self, reward, beta, factor):
        return np.exp(-factor * (beta - reward))


    def get_action_and_targets(self, env):
        self.max_reward_for_net = np.full((self.num_nets), '-inf', dtype=np.float)
        if self.env.unwrapped.spec.id == "Pendulum-v0":
            self.x_init = self.env.env.state
        elif self.env.unwrapped.spec.id == "HumanoidStandup-v2":
            self.x_init = self.env.sim.get_state()
        
        for k in range(self.K):
            self._compute_total_reward(k)
            if self.env.unwrapped.spec.id == "Pendulum-v0":
                self.env.env.state = self.x_init
            elif self.env.unwrapped.spec.id == "HumanoidStandup-v2":
                self.env.sim.set_state(self.x_init)
        print(self.max_reward_for_net)
            
        beta = np.max(self.reward_total)  # maximum reward of all trajectories
        reward_total_non_zero = self._ensure_non_zero(reward=self.reward_total, beta=beta, factor=1/self.lambda_)
        eta = np.sum(reward_total_non_zero)
        omega = 1/eta * reward_total_non_zero
        
        self.U += [np.sum(omega.reshape(len(omega), 1) * self.noise[:, t], axis=0) for t in range(self.T)]
        
        
            
        action = self.U[0]
        
        self.U = np.roll(self.U, -1, axis=0)

        self.U[-1] = self.u_init  #
        self.reward_total[:] = 0
        
        self.noise = np.random.normal(loc=self.noise_mu, scale=self.noise_sigma, size=(self.K, self.T, self.env.action_space.shape[0]))
        
        
        
        return action, self.max_reward_for_net
    
    def control(self, iter=1000):
        for timestamp in range(iter):
            for k in range(self.K):
                self._compute_total_reward(k)

            beta = np.max(self.reward_total)  # maximum reward of all trajectories
#             print()
#             print(self.reward_total)
#             print(beta)
            reward_total_non_zero = self._ensure_non_zero(reward=self.reward_total, beta=beta, factor=1/self.lambda_)
#             print(reward_total_non_zero)
            eta = np.sum(reward_total_non_zero)
            
            omega = 1/eta * reward_total_non_zero
#             print("Omega: {}".format(omega))
#             print("Noise: {}".format(self.noise))
#             print("U before: {}".format(self.U))
            self.U += [np.sum(omega.reshape(len(omega), 1) * self.noise[:, t], axis=0) for t in range(self.T)]
#             print("Incremental: {}".format([np.sum(omega.reshape(len(omega), 1) * self.noise[:, t], axis=0) for t in range(self.T)]))
#             print("U after: {}".format(self.U))
            ############################
            if self.env.unwrapped.spec.id == "Pendulum-v0":
                self.env.env.state = self.x_init
            elif self.env.unwrapped.spec.id == "HumanoidStandup-v2":
                self.env.sim.set_state(self.x_init)
            ############################
            s, r, _, _ = self.env.step(np.array([self.U[0]]))
            try:
                r = r[0]
            except:
                pass
            if self.env.unwrapped.spec.id == "HumanoidStandup-v2":
                r = self._get_reward_from_state(s)
            print("timestamp: {}, action taken: {} reward received: {}".format(timestamp, self.U[0], r))
            if RENDER:
                self.env.render()
#             self.env.sim.render(1024, 1024)

            self.U = np.roll(self.U, -1, axis=0)

            self.U[-1] = self.u_init  #
            self.reward_total[:] = 0
#             print("U after shifting: {}".format(self.U))
#             print("Rewards reset: {}".format(self.reward_total))
            
            ############################
            if self.env.unwrapped.spec.id == "Pendulum-v0":
                self.x_init = self.env.env.state
            elif self.env.unwrapped.spec.id == "HumanoidStandup-v2":
                self.x_init = self.env.sim.get_state()
            ###########################
            
            if self.writer is not None:
                self._write_record(timestamp, r, self.U[0], s)
            
            self.noise = np.random.normal(loc=self.noise_mu, scale=self.noise_sigma, size=(self.K, self.T, self.env.action_space.shape[0]))
    
    def write_record(self, timestamp, reward, action, state):
        action_json = json.dumps(action.tolist())
        state_json = json.dumps(state.reshape(len(state), 1).tolist())
        self.writer.writerow([timestamp, reward, action_json, state_json])
        self.log_file.flush()
        
    def store_state(self, obs, state, rewards):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0

        # replace the old memory with new memory
        index = self.memory_counter % self.memory_size
        self.obs_mem[index] = np.array(obs)
        self.state_mem[index] = state
        self.targets_mem[index] = rewards

        self.memory_counter += 1

In [7]:
num_epos = 1000
for idx in range(num_epos):
    print("Running for episode {}...".format(idx))
    ENV_NAME = "HumanoidStandup-v2"
    TIMESTEPS = 64  # T
    N_SAMPLES = 128  # K
    ACTION_LOW = -1.0
    ACTION_HIGH = 1.0

    # TIMESTEPS = 15 # T
    # N_SAMPLES = 120  # K

    STATE_SAMPLES = 32

    noise_mu = 0
    noise_sigma = 0.2
    lambda_ = 1.25
    gamma = 0.99

    Z = 16

    env = gym.make(ENV_NAME)

    # from gym.wrappers import Monitor
    # env = Monitor(env, './video', force=True)
    # env._max_episode_steps = 200
    # env.render()
    # env.sim.render(1024, 1024)
    print(env.observation_space)
    print(env.action_space)

    U = np.random.uniform(low=ACTION_LOW, high=ACTION_HIGH, size=(TIMESTEPS, env.action_space.shape[0]))  # pendulum joint effort in (-2, +2)
    # print(U)

    log_file = open("polo_record_multiple_{}.tsv".format(idx), "w")

    s = env.reset()

    polo = POLO(env=env, K=N_SAMPLES, T=TIMESTEPS, U=U, lambda_=lambda_, noise_mu=noise_mu, 
                    noise_sigma=noise_sigma, u_init=0, memory_size=512, 
                    observation_space=22, action_space=env.action_space.shape[0],
                    state_space=env.observation_space.shape[0], net_hidden_layers=16, 
                    num_nets=6, state_samples=STATE_SAMPLES, gradient_steps=64, 
                    gamma=gamma, log_file=log_file, noise_gaussian=True)


    # polo.store_state(s[:22], env.sim.get_state())

    rewards = []
    for t in range(1000):
        a, targets = polo.get_action_and_targets(env)
        polo.store_state(s[:22], env.sim.get_state(), targets)
        s, r, _, _ = env.step(np.array([a]))
        rewards.append(r)
        polo.write_record(t, s[0], a, s[:22])
        print("episode: {}, timestamp: {}, action taken: {} reward received: {}".format(idx, t, a, s[0]))
        if RENDER:
            env.render()


        if t != 0 and t % Z == 0 and t >= STATE_SAMPLES:
            print("Updating networks...")
            polo.learn(env)


    # mppi_gym.control(iter=30)

    log_file.close()

Box(376,)
Box(17,)


/home/zhizhen/anaconda3/envs/polo/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/zhizhen/anaconda3/envs/polo/lib/python3.6/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[2.01591893 1.55738503 1.2574061  0.38289886 1.62152857 1.77763039]
timestamp: 0, action taken: [-0.16792404 -0.79324572  0.0767476   0.70023252  0.82098363  0.6449306
 -0.45213628 -0.83539893 -0.15609682 -0.21286734  0.09015993 -0.75705853
  0.48681135 -0.59001441  0.4226868  -0.99859737  0.23199166] reward received: 0.10428574874116879
Creating window glfw
[1.70541435 1.50653618 1.08298179 0.56956718 1.39005444 1.78909673]
timestamp: 1, action taken: [-0.73653857 -0.15586092 -0.14371088 -0.10432229  0.08831533 -0.17644435
  0.07719761 -0.40905663 -0.75561846 -0.64262816 -0.89719175 -0.92814119
 -0.00216634  0.05566629  0.55410835 -0.0583927  -0.89675527] reward received: 0.11031195072479733
[1.84548632 1.71966774 1.1410338  0.48808991 1.52469618 1.85159722]
timestamp: 2, action taken: [ 0.00850785 -0.57781229  0.17043837 -0.06046941  0.14776939  0.931453
  0.55530095  0.85722542 -0.15930335 -0.07451103  0.875281    0.25834101
 -0.08624199 -0.2592055   0.58925958  0.44680039 -0.807133

[2.67585928 2.68749792 2.52952067 1.43703231 2.38485526 2.79571279]
timestamp: 24, action taken: [ 0.63628766  0.36565061  0.5797599  -0.07844764 -0.69229112  0.61019248
  0.0433036  -0.08325643 -0.54813755  0.0387551   0.49713292  0.47695855
  0.44849545 -0.25297567  0.55693223  0.4765762  -0.36096028] reward received: 0.09480816462579031
[3.10581597 3.18449707 2.73391887 2.03846809 2.76515526 3.33187923]
timestamp: 25, action taken: [ 0.28495308 -0.11972451 -1.01428853 -0.01862636  0.28013808 -0.98573628
  0.74263978 -0.10594583  0.93239547  0.08418362 -0.8299873  -0.28278748
 -0.00860428  0.85080588 -0.17154282  0.01096771 -0.80330376] reward received: 0.10776870297067706
[3.7657999  3.72839835 3.00127209 2.94238013 3.49680385 4.28271108]
timestamp: 26, action taken: [ 0.78554173 -0.81084684  0.86116755 -1.0184369  -0.12641682 -0.91480023
  0.21989499  0.03933474 -0.39641892 -0.45655319 -0.93150301  0.18927837
  0.19280025 -1.09951455 -0.02787657 -0.27764047  0.14198105] reward rece

[3.76152551 4.09246638 4.08470006 3.20991087 3.7051543  3.77071073]
timestamp: 48, action taken: [-0.66777267 -0.6222651   0.75154058 -1.01840121  1.10399413  0.01345485
  0.97234803 -0.60099947  0.79754487 -0.80138453 -0.62525414 -0.10768991
  0.03277175 -0.79931113 -0.12893924  0.18707804  0.66541913] reward received: 0.23814392204097598
Updating networks...
[4.0951448  4.10900944 4.23084137 3.58606268 4.06954996 4.1112915 ]
timestamp: 49, action taken: [ 0.70414954  1.06930913 -0.89371902  0.94269735  0.5800998  -0.77450301
  1.13137425 -0.07077066 -0.06548481 -0.63855438  0.94175178  0.38980279
  0.00746451 -0.5664724   0.6490059   0.79822836  0.98648892] reward received: 0.2320329495907414
[3.81604337 3.84019675 3.82833046 3.0590693  3.80446442 3.93764159]
timestamp: 50, action taken: [-0.34090227  1.06025659 -0.04977758  0.39444739 -0.49577142 -0.52079475
  0.75862897 -0.08164882  1.23474222 -0.64612433 -0.82012856 -0.17972465
  0.70952192  0.56675177 -0.5257289  -0.72466948  1.1

...

[35.27341079 35.75836673 35.83891863 35.79135324 35.36434115 35.75029601]
timestamp: 1079, action taken: [ 0.08728788  0.14610913  0.30313662 -0.23190504  0.22653352 -0.09340685
 -0.14508822 -0.08103856  0.17311369  0.13876316 -0.20020567 -0.02420797
  0.08068067 -0.04059016 -0.12085233 -0.19235707 -0.18793307] reward received: 0.46933836493956305
[35.15319201 35.71182238 35.63902123 35.66679237 35.15030857 35.70059677]
timestamp: 1080, action taken: [-0.14743086 -0.20344132  0.20971833 -0.06071808 -0.20426119  0.13978586
  0.01721794  0.04823037 -0.01021663 -0.33570555 -0.15955419 -0.16082105
  0.03794311 -0.14743432 -0.01508657  0.25258465  0.02578055] reward received: 0.46967842823819433
[35.19654635 35.84423679 35.93845334 35.85974941 35.21877047 35.79362078]
timestamp: 1081, action taken: [-0.05242222  0.04810879  0.08480299  0.17671216  0.25785098 -0.074804
 -0.22969056  0.05180244 -0.05976577  0.1503919  -0.08425058 -0.0049115
  0.38079414 -0.08910254  0.13796123 -0.02534234 -0.

[35.57488622 36.16999576 36.0637221  36.10999438 35.52703462 36.13660263]
timestamp: 1103, action taken: [ 0.11547918  0.099434    0.02351114  0.11381286  0.4350844   0.14462675
 -0.19127609  0.01704294  0.12585356 -0.03298168 -0.1319576   0.35873631
  0.01923934 -0.22847861  0.04263448  0.17672653 -0.0794393 ] reward received: 0.4680775215328412
[35.84802111 36.21785486 36.21926437 36.26372505 35.62378486 36.16226847]
timestamp: 1104, action taken: [-0.22577965 -0.0371825   0.07350384  0.23001866 -0.01630552 -0.12293439
 -0.08446863  0.11816287  0.02927766 -0.21573193 -0.06680465 -0.21878947
 -0.17316303 -0.19668374  0.17636808  0.08964997  0.05004446] reward received: 0.46927156934087383
Updating networks...
[35.73621714 36.14480589 36.19906297 36.17243619 35.87808032 36.20458483]
timestamp: 1105, action taken: [-0.19239954 -0.01052803  0.02176736  0.02990649 -0.24601396  0.2623951
 -0.24070891  0.16378746  0.27504339  0.14555126  0.13377576  0.15948296
  0.13869952 -0.01571588  0.12

[35.84791128 36.40460842 36.48955433 36.42711137 36.0503454  36.34717421]
timestamp: 1127, action taken: [-0.11908085 -0.07041623 -0.01699973  0.07267437 -0.26670082  0.11477807
  0.11142887  0.07220742 -0.03939875  0.30113949 -0.12526319 -0.3395102
  0.16998419  0.06797358 -0.04036467 -0.13987938  0.13691274] reward received: 0.48059229658749114
[35.95321947 36.38792368 36.50332498 36.52548815 36.07825441 36.34863717]
timestamp: 1128, action taken: [-0.02698901 -0.31452412  0.00687916 -0.04518502 -0.38168404  0.09514896
 -0.10216176 -0.05780261 -0.0043455  -0.17048017  0.07524024 -0.06364574
  0.21252169 -0.0310143  -0.02495014  0.21488593  0.11052815] reward received: 0.479554209195575
[35.9352793  36.38794021 36.26310562 36.31318429 35.9864547  36.31405773]
timestamp: 1129, action taken: [-0.01433188 -0.17728456 -0.11519754 -0.07480329  0.15798     0.09644905
 -0.19454141 -0.11778278  0.1222467   0.02051908  0.07002011  0.07244779
  0.21488453 -0.04277851 -0.03727434  0.0197224   0.

[35.79186171 36.29043292 36.18412119 36.25179833 35.96107698 36.32035902]
timestamp: 1151, action taken: [ 0.14144575 -0.04998143 -0.06690419  0.10557525 -0.11060919  0.17359926
 -0.0600078  -0.2426008   0.14235593  0.10676828  0.13975489  0.01589737
 -0.18414727  0.15348564  0.11848377 -0.19960435 -0.09628329] reward received: 0.47682169236310495
[35.77694108 36.31726599 36.27526943 36.24547978 35.76994341 36.2917447 ]
timestamp: 1152, action taken: [ 0.20030897  0.12536171  0.05236686 -0.00895522 -0.24532941  0.13607462
 -0.08637425 -0.13420864 -0.00762585  0.22211005 -0.0861291   0.09784579
 -0.00389794 -0.14534977 -0.01971132  0.10536217  0.05179234] reward received: 0.47540866945620536
Updating networks...
[35.80500284 36.37919861 36.291386   36.35096833 35.83040408 36.36547415]
timestamp: 1153, action taken: [-0.04265815 -0.03447314  0.101095    0.01105275 -0.26769024 -0.12934531
  0.10115162 -0.0110333   0.09914357 -0.16190184  0.04342586  0.06425382
 -0.16147916  0.07002436 -0.

NameError: name 'exit' is not defined